In [18]:
%pip install datasets transformers peft trl langchain -q
%pip install accelerate -q
%pip install -i https://pypi.org/simple/ bitsandbytes -q
%pip install evaluate


[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


## Importing Data and Preprocessing

In [2]:
!pip install pynvml

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.1/53.1 kB 2.4 MB/s eta 0:00:00

[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: python -m pip install --upgrade pip


In [3]:
from pynvml import *
nvmlInit()
h = nvmlDeviceGetHandleByIndex(0)
info = nvmlDeviceGetMemoryInfo(h)
print(f'total    : {info.total}')
print(f'free     : {info.free}')
print(f'used     : {info.used}')

total    : 85899345920
free     : 84981776384
used     : 917569536


In [1]:
import pandas as pd
en_de = pd.read_csv('en-de.csv')
en_de = en_de.iloc[:, :2]

en_de.columns = ["en", "de"]

In [18]:
len(en_de)

4447

In [3]:
def choose_prompt_with_target(
    template, src, target, src_lang="English", tgt_lang="Italian"
):
    if template == "ZHANG":
        return f"{src_lang}: {src}\n{tgt_lang}: {target}"
    elif template == "GAO":
        return (f"This is a {src_lang} to {tgt_lang} translation, please provide the {tgt_lang} translation for this sentence: {src}\n{tgt_lang}: {target}")
    elif template == "JIAO":
        return f"Please provide the {tgt_lang} translation for this sentence: {src}\n{tgt_lang}: {target}"
    elif template == "NEW":
      return f"Translate the following {src_lang} sentence into {tgt_lang}:\n {src_lang}{src}\n{tgt_lang}: {target}"
    raise Exception("Choose a valid prompt template. [ZHANG, GAO, JIAO]")

In [4]:
prompts = []

for source, target in en_de[["en", "de"]].values:
    prompts.append(
        choose_prompt_with_target(
            "ZHANG", source, target, src_lang="English", tgt_lang="German"
        )
    )

In [5]:
len(prompts)

4447

In [8]:
for source, target in en_es[["en", "es"]].values:
    prompts.append(
        choose_prompt_with_target(
            "ZHANG", source, target, src_lang="English", tgt_lang="Spanish"
        )
    )

In [6]:
prompts[-1]

"English: Since the first publication of peroral endoscopic myotomy (POEM) by Haruhiro Inoue et al. in 2008 in Japan, various novel endoscopic procedures have been established, which are performed after iatrogenic creation of a submucosal tunnel as a 'new space' 1. Through the artificially formed access in the tela submucosa, interventions in the muscular layer of the esophagus and stomach can be performed while carefully sparing the mucosal layer 2. These include, peroral myotomy of the esophageal muscle layer in patients with achalasia (POEM) and myotomy of the pylorus in patients with gastroparesis (antropyloromyotomy, G-POEM). Further indications include splitting of Zenker diverticulum in POEM technique ('Z-POEM') as well as the removal of subepithelial tumors (STER: submucosal tunneling, endoscopic resection). The long-term therapeutic success (with >\u200a80\u200a% response) of these innovative procedures has now been proven by controlled studies, especially in achalasia 2 3 4 5

In [7]:
from datasets import Dataset, DatasetDict

import random

random.shuffle(prompts)

dataset = DatasetDict(
    {
        "train": Dataset.from_dict({"text": prompts[: int(len(prompts) * 0.80)]}),
        "validation": Dataset.from_dict({"text": prompts[int(len(prompts) * 0.80):]}),
    }
)

In [19]:
dataset['validation']

Dataset({
    features: ['text'],
    num_rows: 890
})

In [122]:
from huggingface_hub import notebook_login

notebook_login()  # same as before

In [10]:
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
import torch

# Define the cache directory for storing model/tokenizer data.
cache_dir = "mistral_cache"

# Specify the model name.
model_name = "mistralai/Mistral-7B-v0.1"

# Configure the model for 4-bit quantization using the BitsAndBytesConfig.
# This is particularly useful for reducing model size and potentially increasing inference speed.
nf4_quant_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=torch.bfloat16,
)

# Load the pre-trained causal language model with the specified quantization configuration.
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="auto",
    quantization_config=nf4_quant_config,
    use_cache=False,
    cache_dir=cache_dir,
)

# Load the tokenizer associated with the model, configuring it for specific use cases.
tokenizer = AutoTokenizer.from_pretrained(
    model_name, cache_dir=cache_dir, add_bos_token=True, add_eos_token=False
)

tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


model.safetensors.index.json:   0%|          | 0.00/25.1k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.94G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

...

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/967 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/72.0 [00:00<?, ?B/s]

In [11]:
import evaluate
def compute_metrics(eval_preds):
    preds, labels = eval_preds
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    bleu = evaluate.load("bleu")
    bleu_score = bleu.compute(predictions=decoded_preds, references=decoded_labels)

    return {"bleu": bleu_score}


In [16]:
def analyze_token_counts(dataset):
    
    token_counts = [len(tokenizer.encode(text)) for text in dataset]
    max_tokens = max(token_counts)
    avg_tokens = sum(token_counts) / len(token_counts)
    
    print(f"Maximum token count: {max_tokens}")
    print(f"Average token count: {avg_tokens:.2f}")

analyze_token_counts(dataset['train'])

TypeError: TextEncodeInput must be Union[TextInputSequence, Tuple[InputSequence, InputSequence]]

In [20]:
from peft import (
    AutoPeftModelForCausalLM,
    LoraConfig,
    get_peft_model,
    prepare_model_for_kbit_training,
)

peft_config = LoraConfig(
    lora_alpha=16, lora_dropout=0.1, r=64, bias="none", task_type="CAUSAL_LM"
)


model = prepare_model_for_kbit_training(model)
model = get_peft_model(model, peft_config)

In [21]:
from transformers import TrainingArguments

output_directory = "clean_creamt_mistral_lora"

training_args = TrainingArguments(
    output_dir=output_directory,
    num_train_epochs=1,
    #max_steps=594,  # comment out to train in epochs
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    warmup_steps=0.03,
    logging_steps=20,
    save_strategy="epoch",
    # evaluation_strategy="epoch",
    evaluation_strategy="steps",
    eval_steps=50,  # comment out this line if you want to evaluate at the end of each epoch
    #eval_accumulation_steps=4,
    learning_rate=2e-3,  # 2e-4 # lower LE for smaller batch sizes
    bf16=True,
    lr_scheduler_type="constant",
)

In [24]:
from trl import SFTTrainer

max_seq_length = 1024  # increase if needed

trainer = SFTTrainer(
                    model=model,
                    peft_config=peft_config,
                    max_seq_length=max_seq_length,
                    tokenizer=tokenizer,
                    packing=True,
                    dataset_text_field="text",
                    args=training_args,
                    train_dataset=dataset["train"],
                    eval_dataset=dataset["validation"],
                    #compute_metrics=compute_metrics,
                  )


Generating train split: 0 examples [00:00, ? examples/s]

Generating train split: 0 examples [00:00, ? examples/s]

In [25]:
trainer.train()

Step,Training Loss,Validation Loss
50,1.368800,1.401202


/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

TrainOutput(global_step=88, training_loss=1.370018254626881, metrics={'train_runtime': 1352.8014, 'train_samples_per_second': 2.079, 'train_steps_per_second': 0.065, 'total_flos': 1.2332138298448282e+17, 'train_loss': 1.370018254626881, 'epoch': 1.0})

In [124]:
from peft import PeftModel, PeftConfig
from transformers import AutoModelForCausalLM, AutoTokenizer
import os


peft_model_path = os.path.join(output_directory, "checkpoint-88")  # change checkpoint path

peftconfig = PeftConfig.from_pretrained(peft_model_path)

model_base = AutoModelForCausalLM.from_pretrained(peftconfig.base_model_name_or_path,
                                             device_map = "auto",
                                             cache_dir = cache_dir
                                            )

tokenizer = AutoTokenizer.from_pretrained(model_name,
                                          cache_dir=cache_dir,
                                          add_bos_token = True,
                                          add_eos_token = False  # always False for inference
                                          )

new_model = PeftModel.from_pretrained(model_base, peft_model_path)

print("Peft model loaded")

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Peft model loaded


In [125]:
new_model.push_to_hub("Ancastal/mistral-en-de")

adapter_model.safetensors:   0%|          | 0.00/109M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/Ancastal/mistral-en-de/commit/7c4125590b2d42fe276ffe681121c672f77880b1', commit_message='Upload model', commit_description='', oid='7c4125590b2d42fe276ffe681121c672f77880b1', pr_url=None, pr_revision=None, pr_num=None)

In [120]:
merged_model = new_model.merge_and_unload()

merged_model.save_pretrained("finetuned-model")
tokenizer.save_pretrained("finetuned-tokenizer")

SafetensorError: Error while serializing: IoError(Os { code: 28, kind: StorageFull, message: "No space left on device" })

In [96]:
def generate_response(prompt, model):
  encoded_input = tokenizer(prompt, return_tensors="pt", add_special_tokens=True)
  model_inputs = encoded_input.to('cuda')

  generated_ids = model.generate(**model_inputs,
                                 max_new_tokens=512,
                                 min_new_tokens=1,
                                 do_sample=False,
                                 pad_token_id=tokenizer.eos_token_id)

  decoded_output = tokenizer.batch_decode(generated_ids)

  return decoded_output[0].replace(prompt, "")


prompt = """English: Cancer is a terrible disease.
German:"""


generate_response(prompt, new_model)

'<s>  Krebs ist eine schreckliche Krankheit.</s>'

In [82]:
test = pd.read_csv('en-de.csv')[:100]
test.columns = ['en', 'de', 'langs']
test.head(2)

,en,de,langs
0,Extracorporeal membrane oxygenation (ECMO) has...,Die extrakorporale Membranoxygenierung (ECMO) ...,en-de
1,The main objective of novel minimally invasive...,Neue minimalinvasive Therapieansätze zur Behan...,en-de


In [83]:
def generate_prompts(df, tgt_lang, prompt_template):
      prompts = []
      for index, row in df.iterrows():
          if prompt_template == "ZHANG23":
            prompt = f'English: {row["en"]}' + '\n' + f'{tgt_lang}:'
          elif prompt_template == "GAO23":
            prompt = f'This is a English to {tgt_lang} translation, please provide the {tgt_lang} translation for this sentence: {row["en"]}' + '\n' + 'Translation:'
          elif prompt_template == "JIAO23":
            prompt = f'Please provide the {tgt_lang} translation for this sentence: {row["en"]}' + '\n' + 'Translation:'
          #elif prompt_template == 'FEWSHOTS':
            #prompt = generate_few_shot_prompt('English', 'Italian', df, row['en'], 5)
          prompts.append(prompt)
      return prompts

# Zero-shot prompting

prompts = generate_prompts(test, "German", "ZHANG23")

In [99]:
prompts[0]

"English: Extracorporeal membrane oxygenation (ECMO) has become a routine method in thoracic surgery. Recent developments in lung transplantation have led to its widespread acceptance. Firstly, ECMO is increasingly being used to bridge patients to transplantation. The best results in this setting have been described with 'awake ECMO'. Secondly, ECMO has replaced cardio-pulmonary bypass as the intraoperative standard device in most centres and is used for treatment and prevention of primary graft dysfunction. Refinements of the devices in use and the cannula design allow an individualised approach tailored to the respiratory and haemodynamic situation of the patients and the anticipated duration of ECMO support.\nGerman:"

In [101]:
generate_response(prompts[0], new_model)

'<s>  Die extrakorporale Membranoxygenierung (ECMO) hat sich in der Thoraxchirurgie zu einem Standardverfahren entwickelt. Neuere Entwicklungen in der Lungentransplantation haben zu einer weitgehenden Akzeptanz der ECMO geführt. Zunächst wird die ECMO zunehmend zur Bridging-Therapie eingesetzt. Hier zeigen sich die besten Ergebnisse mit der „awake ECMO“. Zum zweiten ersetzt die ECMO zunehmend den Herz-Lungen-Bypass als intraoperatives Standardverfahren und wird zur Therapie und Prävention der primären Lungenversagen-Dysfunktion eingesetzt. Neuere Entwicklungen der Geräte und der Kanülengestaltung erlauben eine individualisierte Therapie, die sich an der respiratorischen und haemodynamischen Situation der Patienten und der erwarteten Dauer der ECMO-Unterstützung orientiert.</s>'

In [102]:
results = []
for prompt in prompts:
    results.append(
        generate_response(prompt, new_model)
    )

In [103]:
results_proc = [result.replace("<s>  ", "").replace("<s>", "").replace("</s>","") for result in results]

In [108]:
import evaluate
bleu = evaluate.load('bleu')
chrf = evaluate.load('chrf')
comet = evaluate.load('comet')

Fetching 5 files:   0%|          | 0/5 [00:00<?, ?it/s]

hparams.yaml:   0%|          | 0.00/567 [00:00<?, ?B/s]

.gitattributes:   0%|          | 0.00/1.48k [00:00<?, ?B/s]

README.md:   0%|          | 0.00/3.53k [00:00<?, ?B/s]

LICENSE:   0%|          | 0.00/9.69k [00:00<?, ?B/s]

...

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.10M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/616 [00:00<?, ?B/s]

Encoder model frozen.
/usr/local/lib/python3.10/dist-packages/pytorch_lightning/core/saving.py:188: Found keys that are not in the model state dict but in the checkpoint: ['encoder.model.embeddings.position_ids']


ValueError: Evaluation module inputs are missing: ['sources']. All required inputs are ['sources', 'predictions', 'references']

In [114]:
bleu_score = bleu.compute(predictions=results_proc, references=test['de'][:100])['bleu']
chrf_score = chrf.compute(predictions=results_proc, references=test['de'][:100])['score']
comet_score = comet.compute(predictions=results_proc, references=test['de'][:100], sources=test['en'][:100])['mean_score']

Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/IPython/core/interactiveshell.py", line 3548, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/tmp/ipykernel_712/1645368266.py", line 3, in <module>
    wer_score = evaluate.load('ter').compute(predictions=results_proc, references=test['de'][:100])
  File "/usr/local/lib/python3.10/dist-packages/evaluate/module.py", line 467, in compute
    output = self._compute(**inputs, **compute_kwargs)
  File "/root/.cache/huggingface/modules/evaluate_modules/metrics/evaluate-metric--ter/9c9af3214842a93c26c9ac10ddc5d07559df8e38c0ab3b599e8121b9aae196bd/ter.py", line 210, in _compute
    output = sb_ter.corpus_score(predictions, transformed_references)
  File "/usr/local/lib/python3.10/dist-packages/sacrebleu/metrics/base.py", line 417, in corpus_score
    stats = self._extract_corpus_statistics(hypotheses, references)
  File "/usr/local/lib/python3.10/dist-packages/sacrebleu/metrics/base.py"

In [113]:
bleu_score, chrf_score,comet_score, wer_score

(0.21742658148983116, 53.65640517355328, 0.8402795797586441)

In [119]:
model.push_to_hub("Ancastal")

HfHubHTTPError:  (Request ID: Root=1-663d19df-7653da6d4f71823d1aa447c5;731e4dd2-72f9-4404-aba9-44b73e979df5)

403 Forbidden: You don't have the rights to create a model under the namespace "Ancastal".
Cannot access content at: https://huggingface.co/api/repos/create.
If you are trying to create or update content,make sure you have a token with the `write` role.